# Notebook to tidy 1999 Referenda Data

Thanks to Zach A for repeatedly nudging me to do this

Thanks to David Michael for the data

## Libraries

In [198]:
import pandas as pd
import xlrd

filepath = '1999_referenda/results_by_polling_place/v4_3ADEL.XLS'

## Functions

#### `left_of_bracket(s)`
* takes string
    * if string contains left bracket, returns whatever occurs before the bracket
    * if string doesn't contain left bracket, returns whole string

In [199]:
def left_of_bracket(s):
    if '(' in s:
        needle = s.find('(')
        r = s[:needle-1].strip()
        return r
    else:
        return s

In [200]:
print(left_of_bracket('Croydon Park'))
print(left_of_bracket('Croydon Park West (Adelaide)'))

Croydon Park
Croydon Park West


#### `referenda_by_electorate_get_metadata(filepath)`
* takes filepath, string
* get properties of data, state, electorate, et al

In [201]:
def referenda_by_electorate_get_metadata(filepath):
    x = xlrd.open_workbook(filepath)
    x_sheet = x.sheet_by_index(0)

    date = x_sheet.cell(0,0).value.strip()
    referenda = x_sheet.cell(5,0).value.strip()
    state = x_sheet.cell(8,0).value.strip()
    electorate = x_sheet.cell(9,0).value.strip()
    
    r = dict(
        date = date,
        referenda = referenda,
        state = state,
        electorate = electorate
    )
    
    return r

In [202]:
referenda_by_electorate_get_metadata(filepath)

{'date': 'Referendum: 6 November 1999',
 'electorate': 'Adelaide (86272)',
 'referenda': 'the Constitutional Alteration (Establishment of Republic) 1999',
 'state': 'South Australia'}

#### `split_electorate_enrolment(s)`
* takes string containing electorate and enrolment
    * Field in the form: `"Adelaide (86272)"`
* splits them, returns dict containing both

In [203]:
def split_electorate_enrolment(s):
    
    # splitting text in the form 'Adelaide (86272)'
    needle = s.find('(')
    
    enrolment = s[needle:].strip() # take right of needle
    enrolment = enrolment[1:-1] # remove first and last chars, ie, remove brackets
    electorate = s[:needle-1].strip() # take everything from the start to one place before the bracket
    
    r = dict(
        electorate = electorate,
        enrolment = enrolment
    )
    
    return r

#### `state_abbreviation(state)`
* Converts South Australia to SA, et al

In [204]:
def state_abbreviation(state):
    spaces = state.count(' ')
    
    if spaces == 2:
        bits = state.split(' ')
        r=''
        for b in bits:
            r = r + b[:1].upper() # for each word in state grab first letter
        return r
    elif 'Australia' in state:
        r = state[:1].upper() + 'A'
        return r
    elif state == 'Queensland':
        return 'QLD'
    elif state == 'Northern Territory':
        return 'NT'
    else:
        r = state[:3].upper()
        return r

In [205]:
print(state_abbreviation('New South Wales'))
print(state_abbreviation('Victoria'))
print(state_abbreviation('Queensland'))
print(state_abbreviation('South Australia'))
print(state_abbreviation('Western Australia'))
print(state_abbreviation('Tasmania'))
print(state_abbreviation('Northern Territory'))
print(state_abbreviation('Australian Capital Territory'))

NSW
VIC
QLD
SA
WA
TAS
NT
ACT


#### `tidy_referenda_date(s)`
* takes string containing referenda date
    * Field in the form: `"Referendum: 6 November 1999"`
* returns datetime in the form `1999-11-06 00:00:00`

In [206]:
def tidy_referenda_date(s):
    # date in the form "Referendum: 6 November 1999"
    needle = s.find(':')
    needle = needle + 1 # skip the semicolon and the space
    s = s[needle:].strip()

    # parse as date
    r = pd.to_datetime(s)
    
    return r

In [207]:
print(tidy_referenda_date('Referendum: 6 November 1999'))

1999-11-06 00:00:00


#### `referenda_by_electorate_tidy_metadata(metadata)`
* takes metadata, dict, containing state, electorate, referenda and date keys
* runs separate tidy up functions properties of data, state, electorate, et al

In [208]:
def referenda_by_electorate_tidy_metadata(metadata):
    electorate = metadata['electorate']
    state = metadata['state']
    date = metadata['date']
    
    # tidy electorate, get enrolment
    electorate_enrolment = split_electorate_enrolment(electorate)
    
    metadata['electorate'] = electorate_enrolment['electorate']
    metadata['enrolment'] = electorate_enrolment['enrolment']
    
    # tidy date
    date = tidy_referenda_date(date)
    
    metadata['date'] = date
    
    # tidy state
    state = state_abbreviation(state)
    
    metadata['state'] = state
    
    return metadata

In [209]:
metadata = {
    'date': 'Referendum: 6 November 1999',
    'referenda': 'the Constitutional Alteration (Establishment of Republic) 1999',
    'state': 'South Australia',
    'electorate': 'Adelaide (86272)'
}

print('Before:')
print(metadata)

metadata = referenda_by_electorate_tidy_metadata(metadata)

print('After:')
print(metadata)

Before:
{'date': 'Referendum: 6 November 1999', 'referenda': 'the Constitutional Alteration (Establishment of Republic) 1999', 'state': 'South Australia', 'electorate': 'Adelaide (86272)'}
After:
{'date': Timestamp('1999-11-06 00:00:00'), 'referenda': 'the Constitutional Alteration (Establishment of Republic) 1999', 'state': 'SA', 'electorate': 'Adelaide', 'enrolment': '86272'}


#### `referenda_by_electorate_get_data(filepath)`
* takes filepath
* returns data

In [210]:
def referenda_by_electorate_get_data(filepath):
    # define headers for dataframe
    columnheaders = [
        'polling_place_raw',
        'yes_or_no',
        'yes_n',
        'yes_p',
        'no_n',
        'no_p',
        'formal_n',
        'formal_p',
        'informal_n',
        'informal_p',
        'total_n',
        'total_p'
    ]
    
    # read file
    df_result = pd.read_excel(
        filepath,
        skiprows = 13, # the data starts on the 14th row
        names=columnheaders
    )
    
    # percentages are represented by 61.72 rather than 0.6172, this fixes that
    
    lambda_fix_percentages = lambda x: x/100

    # run on the relevant columns
    df_result['yes_p'] = df_result['yes_p'].apply(lambda_fix_percentages)
    df_result['no_p'] = df_result['no_p'].apply(lambda_fix_percentages)
    df_result['formal_p'] = df_result['formal_p'].apply(lambda_fix_percentages)
    df_result['informal_p'] = df_result['informal_p'].apply(lambda_fix_percentages)
    df_result['total_p'] = df_result['total_p'].apply(lambda_fix_percentages)

    # polling places in >1 electorate are represented like "Croydon (Adelaide)"
    # this creates a poling place column, so the frame can be combined for mapping

    lambda_polling_places = lambda x: left_of_bracket(x)

    # use above function to make a polling place column, i.e, remove ' (Adelaide)' from 'Enfield North (Adelaide)'
    df_result['polling_place'] = df_result['polling_place_raw'].apply(lambda_polling_places)
    
    return df_result

In [211]:
filepath = '1999_referenda/results_by_polling_place/v4_3ADEL.XLS'
print(referenda_by_electorate_get_data(filepath).head(3))

   polling_place_raw yes_or_no  yes_n   yes_p  no_n    no_p  formal_n  \
0      Adelaide East       Yes    465  0.6700   229  0.3300       694   
1  Adelaide Hospital       Yes    187  0.6172   116  0.3828       303   
2     Adelaide South       Yes   1056  0.6467   577  0.3533      1633   

   formal_p  informal_n  informal_p  total_n  total_p      polling_place  
0    0.9914           6      0.0086      700   0.0086      Adelaide East  
1    0.9806           6      0.0194      309   0.0038  Adelaide Hospital  
2    0.9903          16      0.0097     1649   0.0202     Adelaide South  


#### `referenda_by_electorate_add_metadata(data, metadata)`
* takes data frame for electorate result
* adds chosen metadata

In [221]:
def referenda_by_electorate_add_metadata(data, metadata):

    # add metadata columns as desired
    data['state'] = metadata['state']
    data['electorate'] = metadata['electorate']
    #data['enrolment'] = metadata['enrolment']
    #data['date'] = metadata['date']
    #data['referenda'] = metadata['referenda']
    
    # reorder columns - if you uncomment above, make sure to include below also
    data = data[[
        #'referenda',
        #'date',
        'state',
        'electorate',
        #'enrolment',
        'polling_place',
        'polling_place_raw',
        'yes_or_no',
        'yes_n',
        'yes_p',
        'no_n',
        'no_p',
        'formal_n',
        'formal_p',
        'informal_n',
        'informal_p',
        'total_n',
        'total_p'
    ]]

    return data

In [222]:
# get file
filepath = '1999_referenda/results_by_polling_place/v4_3ADEL.XLS'

# get and tidy metadata
metadata = referenda_by_electorate_get_metadata(filepath)
metadata = referenda_by_electorate_tidy_metadata(metadata)

# get data
data = referenda_by_electorate_get_data(filepath)

# append metadata to data
data = referenda_by_electorate_add_metadata(data, metadata)

data.head(3)

,state,electorate,polling_place,polling_place_raw,yes_or_no,yes_n,yes_p,no_n,no_p,formal_n,formal_p,informal_n,informal_p,total_n,total_p
0,SA,Adelaide,Adelaide East,Adelaide East,Yes,465,0.6700,229,0.3300,694,0.9914,6,0.0086,700,0.0086
1,SA,Adelaide,Adelaide Hospital,Adelaide Hospital,Yes,187,0.6172,116,0.3828,303,0.9806,6,0.0194,309,0.0038
2,SA,Adelaide,Adelaide South,Adelaide South,Yes,1056,0.6467,577,0.3533,1633,0.9903,16,0.0097,1649,0.0202


## Get Republic

#### Find files

In [234]:
# libraries I need
import os
from os import listdir
from os.path import isfile, join

# get all filenames from dir
filedir = '1999_referenda/results_by_polling_place/'
files = [f for f in listdir(filedir) if isfile(join(filedir, f))]

the_files_we_want = []

# in the worlds laziest naming scheme, 3ASTO.* is the republic, 3ASTO_2 is constitutional recognition
for f in files:
    if ('XLS' in f) & ('_2' not in f):
        the_files_we_want.append(f)

print(the_files_we_want[:3])

['v4_3ADEL.XLS', 'v4_3ASTO.XLS', 'v4_3BALL.XLS']


In [223]:
files = ['v4_3ADEL.XLS','v4_3MCEW.XLS']
filedir = '1999_referenda/results_by_polling_place/'

for f in files:
    filepath = filedir + '/' + f

    metadata = referenda_by_electorate_get_metadata(filepath)
    metadata = referenda_by_electorate_tidy_metadata(metadata)
    
    data = referenda_by_electorate_get_data(filepath)
    
    data = referenda_by_electorate_add_metadata(data, metadata)
    print(data.head(3))

  state electorate      polling_place  polling_place_raw yes_or_no  yes_n  \
0    SA   Adelaide      Adelaide East      Adelaide East       Yes    465   
1    SA   Adelaide  Adelaide Hospital  Adelaide Hospital       Yes    187   
2    SA   Adelaide     Adelaide South     Adelaide South       Yes   1056   

    yes_p  no_n    no_p  formal_n  formal_p  informal_n  informal_p  total_n  \
0  0.6700   229  0.3300       694    0.9914           6      0.0086      700   
1  0.6172   116  0.3828       303    0.9806           6      0.0194      309   
2  0.6467   577  0.3533      1633    0.9903          16      0.0097     1649   

   total_p  
0   0.0086  
1   0.0038  
2   0.0202  
  state electorate  polling_place polling_place_raw yes_or_no  yes_n   yes_p  \
0   VIC     McEwen  Arthurs Creek     Arthurs Creek        No    148  0.4582   
1   VIC     McEwen         Avenel            Avenel        No    206  0.3515   
2   VIC     McEwen   Badger Creek      Badger Creek        No    518  0.4105  